In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.lqns.*
import jline.solvers.ln.*
import jline.solvers.nc.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.SILENT)

// Create model
val model = LayeredNetwork("myLayeredModel")

// Definition of processors, tasks and entries
val P1 = Processor(model, "P1", 1, SchedStrategy.PS)
val T1 = Task(model, "T1", 100, SchedStrategy.REF).on(P1)
val E1 = Entry(model, "E1").on(T1)

val P2 = Processor(model, "P2", 1, SchedStrategy.PS)
val T2 = Task(model, "T2", 1, SchedStrategy.INF).on(P2)
val E2 = Entry(model, "E2").on(T2)
val E3 = Entry(model, "E3").on(T2)

In [ ]:
// Definition of activities
T1.setThinkTime(Erlang.fitMeanAndOrder(10.0, 1))

val A1 = Activity(model, "A1", Exp(1.0)).on(T1).boundTo(E1).synchCall(E2).synchCall(E3, 1.0)

val A20 = Activity(model, "A20", Exp(1.0)).on(T2).boundTo(E2)
val A21 = Activity(model, "A21", Exp(1.0)).on(T2)
val A22 = Activity(model, "A22", Exp(1.0)).on(T2).repliesTo(E2)
T2.addPrecedence(ActivityPrecedence.Serial(A20, A21, A22))

val A3 = Activity(model, "A3", Exp(1.0)).on(T2).boundTo(E3).repliesTo(E3)

In [ ]:
// Instantiate LQNS solver
val options = LQNS.defaultOptions()
options.keep = true
options.verbose = 0

if (LQNS.isAvailable()) {
    try {
        val solverLqns = LQNS(model, options)
        val avgTableLqns = solverLqns.avgTable
        println("LQNS Solver Results:")
        avgTableLqns.print()
    } catch (e: Exception) {
        println("LQNS solver error: ${e.message}")
    }
} else {
    println("LQNS solver not available")
}

In [ ]:
// LN solver with NC
val lnoptions = LN.defaultOptions()
lnoptions.verbose = 0
val solveroptions = NC.defaultOptions()
solveroptions.verbose = 0

try {
    val solverNc = LN(model) { l -> NC(l, solveroptions) }
    solverNc.setOptions(lnoptions)
    val avgTableNc = solverNc.avgTable
    println("\nLN with NC Solver Results:")
    avgTableNc.print()
} catch (e: Exception) {
    println("LN with NC solver error: ${e.message}")
}